In [2]:
# Install llama-cpp-python.
!pip install llama-cpp-python

In [ ]:
# Import guidance.
import guidance

In [4]:
# Create a LlamaCppSettings instance and set the model path and tokenizer name.
settings = guidance.llms.LlamaCppSettings()
settings.model = "../ggml-v2-models/WizardLM-7B-uncensored.ggml.q5_1.bin"
# Hugginface original tokenizer needed at the moment.
settings.tokenizer_name = "ehartford/WizardLM-7B-Uncensored"

# Other models would look like this:
# settings.model = "../ggml-v3-models/gpt4-x-vicuna-13B.ggmlv3.q5_1.bin"
# settings.tokenizer_name = "TheBloke/gpt4-x-vicuna-13B-HF"

# Or like this:
# settings.model = "../ggml-v3-models/Manticore-13B.ggmlv3.q5_1.bin"
# settings.tokenizer_name = "openaccess-ai-collective/manticore-13b"

# Set additional arguments like number of threads and gpu layers.
settings.n_gpu_layers = 10
settings.n_threads = 12

In [5]:
# Create a LlamaCpp instance and pass the settings to it.
llama = guidance.llms.LlamaCpp(settings=settings)

ValueError: Model path does not exist: ../ggml-v2-models/WizardLM-7B-uncensored.ggml.q5_1.bin

In [6]:
# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
character_maker = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "id": "{{id}}",
    "description": "{{description}}",
    "name": "{{gen 'name' temperature=0.7}}",
    "age": {{gen 'age' pattern='[0-9]+' max_tokens=2 stop=','}},
    "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    "weapon": "{{select 'weapon' options=valid_weapons}}",
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra' temperature=0.7}}",
    "strength": {{gen 'strength' pattern='[0-9]+' max_tokens=2 stop=','}},
    "items": [{{#geneach 'items' num_iterations=5 join=', '}}"{{gen 'this' temperature=0.7}}"{{/geneach}}]
}```""")

# generate a character
char = character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons, llm=llama
)

print(char)

NameError: name 'llama' is not defined

In [ ]:
# define a recursive function that takes a jsonformer schema and returns a guidance program
def jsonformer2guidance(schema, key=None, indent=0):
    out = ""
    if schema['type'] == 'object':
        out += "  "*indent + "{\n"
        for k,v in schema['properties'].items():
            out += "  "*(indent+1) + k + ": " + jsonformer2guidance(v, k, indent+1) + ",\n"
        out += "  "*indent + "}"
        return out
    elif schema['type'] == 'array':
        if 'max_items' in schema:
            extra_args = f" max_iterations={schema['max_items']}"
        else:
            extra_args = ""
        return "[{{#geneach '"+key+"' stop=']'"+extra_args+"}}{{#unless @first}}, {{/unless}}" + jsonformer2guidance(schema['items'], "this") + "{{/geneach}}]"
    elif schema['type'] == 'string':
        return "\"{{gen '"+key+"' stop='\"'}}\""
    elif schema['type'] == 'number':
        return "{{gen '"+key+"' pattern='[0-9\\.]' stop=','}}"
    elif schema['type'] == 'boolean':
        return "{{#select '"+key+"'}}True{{or}}False{{/select}}"

In [ ]:
# define a jsonformer schema
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "max_items": 6,
            "items": {"type": "string"}
        }
    }
}



In [ ]:
# run the guidance program
prompt = "Generate a person's information based on the following schema:"
program = guidance(prompt + "\n" + jsonformer2guidance(json_schema))
out = program(llm=llama)
print(out)

In [ ]:
settings =  guidance.llms.LlamaCppSettings()
settings.model = "../ggml-v3-models/Manticore-13B-Chat-Pyg.ggmlv3.q5_1.bin"
settings.n_gpu_layers = 10
settings.n_threads = 12
settings.tokenizer_name = "openaccess-ai-collective/manticore-13b-chat-pyg"
settings.before_role = "<|"
settings.after_role = "|>"
llama = guidance.llms.LlamaCpp(settings=settings, caching=False)


In [ ]:

# llama = guidance.llms.LlamaCpp(settings=settings, caching=False)
experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}
''', llm=llama)



In [ ]:
print(experts(query='How can I be more productive?', llm=llama))